In [1]:
import json
from collections import defaultdict

from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, clear_output
import ipywidgets as widgets
import plotly.graph_objs as go
import plotly
import numpy as np
import numpy.linalg as nla

import sim_lib.util as util

init_notebook_mode(connected=True)

In [2]:
_SIMPLEST = False

filename = 'ntwk_data/ntwk_data.json'
if _SIMPLEST:
    filename = 'ntwk_data/ntwk_data_simplest.json'

In [3]:
ntwk_data = {}
with open(filename, 'r') as ntd:
    ntwk_data = json.loads(ntd.read())

In [4]:
class NtwkCompVisLV:
    def __init__(self, data):
        
        self.data = data

        r_vals = list(self.data.keys())[::-1]
        p_vals = list(self.data[r_vals[0]].keys())[::-1]
        
        # Welfare by p
        self.p_val = widgets.Select(
                                    options=p_vals,
                                    description='Prob info trans',
                                    disabled=False)
        self.r_val = widgets.Select(
                                    options=r_vals,
                                    description='Resources per vertex',
                                    disabled=False)
        
        self.radius = widgets.IntSlider(
                                    value=3,
                                    min=1,
                                    max=10,
                                    step=1,
                                    description='Radius of local ball',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal')

        self.params = widgets.SelectMultiple(
                                    options=['mean', 'std', 'min', 'max', 'size'],
                                    description='Params of local utils',
                                    disabled=False)
        self.ntwk_types = widgets.SelectMultiple(
                                    options=['rl', 'ws', 'er', 'cm', 'kg'],
                                    description='Network type',
                                    disabled=False)
        
        self.final_utils = widgets.ToggleButton(
                                    value=False,
                                    description='Use final utils',
                                    disabled=False,
                                    button_style='',
                                    tooltip='Else use initial')

        # Update button
        self.plot_button = widgets.Button(
                                    description='Plot',
                                    disabled=False,
                                    button_style='',
                                    tooltip='Plots by selected parameters')
        self.plot_button.on_click(self.plot)
        
        self.visuals = [self.p_val, self.r_val, self.radius, self.params,
                        self.ntwk_types, self.final_utils, self.plot_button]
        
        self.render_visuals()
        
    def render_visuals(self):

        # Render widgets
        for vis in self.visuals:
            display(vis)

    def plot(self, button):
        
        # Reset widgets
        clear_output()
        self.render_visuals()
        
        p_val = self.p_val.value
        r_val = self.r_val.value
        radius = self.radius.value
        
        params = self.params.value
        
        ntwk_types = self.ntwk_types.value
        
        use_final_utils = self.final_utils.value

        # ntwk_type : [ [ { std, mean, max, min, size } per vtx ] per sim run ]
        local_utils = {}
        
        for ntwk in ntwk_types:
            use_data = self.data[r_val][p_val]
            
            local_utils[ntwk] = []
            
            graphs = use_data['init_graphs'] if use_final_utils else use_data['graphs']
            for isg in graphs[ntwk]:                
                init_g = util.json_to_graph(isg)
                
                local_utils[ntwk].append([])
                
                for vtx in init_g.vertices:
                    # Get radius ball
                    ball_vtxs = set()
                    queue = [(vtx, 0)]
                    
                    while queue:
                        cur_vtx, dist = queue[0]
                        queue.pop(0)
                        
                        if dist == radius:
                            continue
                        
                        for nbor in cur_vtx.nbors:
                            if nbor not in ball_vtxs:
                                queue.append((nbor, dist + 1))
                                ball_vtxs.add(nbor)
                                
                    bv_utils = [ bv.utility for bv in ball_vtxs ]
                    bv_stats = { 'mean' : np.mean(bv_utils),
                                 'std' : np.std(bv_utils), 
                                 'max' : np.max(bv_utils),
                                 'min' : np.min(bv_utils),
                                 'size': len(bv_utils) }
                    local_utils[ntwk][-1].append(bv_stats)
        
        # Plot local util stats
        fig = go.Figure()

        colors = plotly.colors.DEFAULT_PLOTLY_COLORS
        
        i_idx = 0
        for ntwk, all_run_stats in local_utils.items():
            
            indiv_fig = go.Figure()
            
            for j_idx, param in enumerate(params):
                
                all_param_vals = []
                
                for run_stats in all_run_stats:
                    color = colors[(i_idx + j_idx) % len(colors)]

                    param_vals = [ rs[param] for rs in run_stats ]
                    all_param_vals.extend(param_vals)

                fig.add_trace(go.Histogram(x=all_param_vals,
                                         marker_color=color,
                                         opacity=0.75,
                                         name=f'ntwk={ntwk} param={param}'))

                indiv_fig.add_trace(go.Histogram(x=all_param_vals,
                             marker_color=color,
                             opacity=0.75,
                             name=f'param={param}'))
                    
            indiv_fig_title = f'{ntwk} Local Distribution Measures (p={p_val}, r={r_val})'
            indiv_fig.layout.update(title=indiv_fig_title,
                        yaxis=dict(title='Param values'),
                        barmode='overlay',
                        plot_bgcolor='rgba(0,0,0,0)')
            iplot(indiv_fig)
            
            i_idx += 1

        plot_title = f"Local Distribution Measures (comparison)"
        fig.layout.update(title=plot_title, showlegend=True,
            plot_bgcolor='rgba(0,0,0,0)', barmode='overlay')

        iplot(fig)


In [5]:
vis = NtwkCompVisLV(ntwk_data)

Select(description='Prob info trans', options=('0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', …

Select(description='Resources per vertex', options=('2', '4', '8', '16', '32', '64', '128'), value='2')

IntSlider(value=3, continuous_update=False, description='Radius of local ball', max=10, min=1)

SelectMultiple(description='Params of local utils', options=('mean', 'std', 'min', 'max', 'size'), value=())

SelectMultiple(description='Network type', options=('rl', 'ws', 'er', 'cm', 'kg'), value=())

ToggleButton(value=False, description='Use final utils', tooltip='Else use initial')

Button(description='Plot', style=ButtonStyle(), tooltip='Plots by selected parameters')